In [ ]:
from datetime import datetime
from datetime import date
from math import pi
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

path = os.getcwd()
print(path)

# import API
from pycoingecko import CoinGeckoAPI

#import py scripts
import sandinfo as si
import crypto_ETL as ce

# pandas options
pd.options.display.float_format = '{:,.2f}'.format
pd.options.mode.chained_assignment = None

# USE CAUTION/UNCOMMENT if necessary
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 150)

# plotting
%matplotlib inline

# Constants:
SAND_PRICE_DATETIME = '2021-11-24' # Date cut for analysis
SAND_PRICE_CHANGEPT = '2021-10-28' # Date Facebook announces Meta
SAND_PRICE_IN_USD = 7.23 # Price of SAND at date cutoff
RISK_FREE_RATE = 0.025


## Data Ingestion

In [ ]:
# Initial dataframe is the top-level, need to flatten
df_initial = pd.read_json('./final_jsons/top_wallets_and_transactions_with_USD_prices_and_tags.json')
display(df_initial.head())
print(df_initial.shape)
print(df_initial.columns)

# df_flat takes the json object and flattens for transaction level data.
df_flat = ce.flat_sand_data("./final_jsons/top_wallets_and_transactions_with_USD_prices_and_tags.json")

In [ ]:
# List of unique "tags". Wallets are "tagged" to denote a Smart Contract or belonging to a centralized exchange.
# For the purposes of this study in which we are assessing the investment activity performance, we are excluding tagged wallets.

df_flat['holders_tag'].unique()

In [ ]:
# df_no_tag filters out the all transactions with holders that are "tagged".

df_no_tag = df_flat[df_flat['holders_tag'].isnull()]

In [ ]:
# Creates a new DF that allows for easier readiblity such as converting UNIX timestamp to datetime.

ce.make_readible(df_no_tag)

## Most frequent From and To Wallet Addresses:

In [ ]:
display(df_no_tag['address_from'].value_counts())
display(df_no_tag['address_to'].value_counts())


In [ ]:
# df_tx takes a copy of df_no_tag and makes for better readability.
# EXCLUDES tagged wallets.

df_tx = ce.make_readible(df_no_tag)
df_tx.columns

## Wallet-Level Data:

In [ ]:
# wallet_level_data (df): this should provide a wallet-level dataframe to house investment return data.
# We are choosing df_flat which still has the UNIX timestamp.

wallet_level_data = ce.aggregate_wallet(df_flat,df_tx)
wallet_level_data

In [ ]:
# wallet_level_data2 collapses multiple initial transactions into single aggregate purchases.

wallet_level_data2 = ce.collapse_initial(wallet_level_data)
wallet_level_data2 = wallet_level_data2.reset_index() 
wallet_level_data2['holders_address'].nunique()

### Now time to get the transaction for each wallet to calculate the end value....

In [ ]:
# df_tx[df_tx['holders_address']]
df_tx_reduce = df_tx[['holders_address','date_time','value','USD_price_at_timestamp','value_in_USD']]
df_tx_reduce

### For later, all these wallets started off with OUTFLOWS and are skewing our analysis

In [ ]:
display(df_tx_reduce.groupby('holders_address').last().reset_index().sort_values('value',axis=0,ascending=False)[0:5])
print('Let\'s drop the top 5 of these transaction from analysis.')

drop_address = df_tx_reduce.groupby('holders_address').last().reset_index().sort_values('value',axis=0,ascending=False)[0:5]['holders_address'].tolist()
drop_address

In [ ]:
df_wallet_return = df_tx_reduce.copy()
df_wallet_return = df_wallet_return.groupby('holders_address').sum()[['value','value_in_USD']].reset_index()
df_wallet_return.sort_values('value',axis=0,ascending=False,inplace=True)
df_wallet_return

In [ ]:
### the value is how many coins are left in the value
### the value_in_USD is their weighted average investment base

df_wallet_return['ending_value_in_USD'] = df_wallet_return['value']*SAND_PRICE_IN_USD
df_wallet_return['return_by_wallet'] = df_wallet_return['ending_value_in_USD']/df_wallet_return['value_in_USD']-1
df_wallet_return.describe()

In [ ]:
display(df_wallet_return.sort_values('return_by_wallet',ascending=False))

In [ ]:
df_wallet_return.hist('return_by_wallet')
plt.show()

print('Number of wallets with POSITIVE returns:', len(df_wallet_return[df_wallet_return['return_by_wallet']>0]))
print('Number of wallets with NEGATIVE returns:', len(df_wallet_return[df_wallet_return['return_by_wallet']<0]))

In [ ]:
df_wallet_return_merge = df_wallet_return[['holders_address','ending_value_in_USD','return_by_wallet']]
df_wallet_return_merge

In [ ]:
wallet_level_data2 = pd.merge(wallet_level_data2, df_wallet_return_merge, on ='holders_address', how ='inner')
wallet_level_data2.head()

# Sanity Checks

In [ ]:
print(df_tx.columns)

# Only pulls in transaction type: transfer.
print(f'\n\nThe type of transaction pulled:')
display(df_tx.value_counts(['tx_type']))

# Date Range, seems reasonable
print(f'\n\nThe first date of date range is:')
display(df_tx['date_time'].min())
print(f'\n\nThe last date of date range is:')
display(df_tx['date_time'].max())

# Any same to/from addresses?
print(f'\n\nWere there transactions where the tokens were transfer from one wallet to itself?')
display(df_tx.value_counts(df_tx['address_from']==df_tx['address_to']))
                       
# 18k transactions in line with Ethan's estimates.
print(f'\n\nThe total number of transactions in the dataset:')
print(f'Shape: {df_tx.shape}')


In [ ]:
# The sum of the holders_share should be less than 100%
print(f'\n\nThe sum of the holders_share is:')
total_share = df_tx[['holders_address', 'holders_share']].groupby('holders_address').max().sort_values('holders_share', ascending=False)
display(total_share['holders_share'].sum())

# What is the sum of holders_share that are "Smart Contracts?"
print(f'\n\nThe sum of the holders_share tagged as "Smart Contracts" is:')
tagged_transactions = df_flat[df_flat['holders_tag'].notnull()]
smart_contracts = tagged_transactions[tagged_transactions['holders_tag'].str.contains('Smart Contract',regex=False)]
smart_contracts_share = smart_contracts[['holders_address', 'holders_share','holders_tag']].groupby('holders_address').max().sort_values('holders_share', ascending=False).sum()
display(smart_contracts_share['holders_share'].sum().round())

# What is the sum of holder_share that are Exchanges?
print(f'\n\nThe sum of the holders_share tagged as exchanges is:')
not_smart_contracts = tagged_transactions[~tagged_transactions['holders_tag'].str.contains('Smart Contract',regex=False)]
not_contracts_share = not_smart_contracts[['holders_address', 'holders_share','holders_tag']].groupby('holders_address').max().sort_values('holders_share', ascending=False).sum()
display(not_contracts_share['holders_share'].sum())

# The sum of the balance should be less than 100% of the total # of circulating tokens
print(f'\n\nThe sum of the holders_balance is:')
total_balance = df_tx[['holders_address', 'holders_balance']].groupby('holders_address').max().sort_values('holders_balance', ascending=False)
display(total_balance['holders_balance'].sum().round())


# The USD price at each date_time stamp
print(f'\n\nThe USD price at each date_time stamp:')
display(df_tx[['date_time', 'USD_price_at_timestamp']].sort_values('date_time'))


## Daily SAND price, market cap, and trading volume

In [ ]:
# Graph full time range of $SAND returns for COIN Gecko API from 2020-08-05 to now
# Set time range, then pull SAND returns, and lastly graph the dataset
time_range = si.set_timeframe('2020-8-5', '2021-12-1')
sand_info_df = si.get_sand_returns(time_range)
si.sand_price_volume_plot(sand_info_df)

# Graph more recent time range of $SAND returns with local max and min
time_range_m = si.set_timeframe('2021-8-5', '2021-12-1')
sand_info_m = si.get_sand_returns(time_range_m)
si.sand_mm_plot(sand_info_m)


In [ ]:
# Shows the local min and max values in the $SAND price data
max_min_df = sand_info_df.loc[(sand_info_df['min'].notnull()) | (sand_info_df['max'].notnull())]

In [ ]:
sand_info_df

## Calculate SAND returns and alpha

In [ ]:
# SANITY CHECK to see if the dates for the two df (wallet_level_data2 and sand_info_df) align

# From this check, it's evident that Coin Gecko does not have data on the $SAND price
# before 8/14/2021, which means we won't be able to pull the price data for any dates before then.

print("The minimum date in 'wallet_level_data2':\n", wallet_level_data2['initial_dt'].min(), "\n\n")
print("The minimum date in 'sand_info_df' that is pulled from coin gecko is:\n\n", sand_info_df.min())



In [ ]:
# Calculating SAND returns and alpha

wallet_level_data2['sand_return'] = SAND_PRICE_IN_USD/wallet_level_data2['USD_price_at_timestamp']-1
wallet_level_data2['wallet_alpha'] = wallet_level_data2['return_by_wallet'] - wallet_level_data2['sand_return']
wallet_level_data2

## Calculate both annualized wallet and SAND returns

In [ ]:
# Calculate wallet returns
wallet_level_data2['annualized_wallet_returns'] = (1+wallet_level_data2['return_by_wallet'])\
                                                    **(365/wallet_level_data2['holding_period_days'])-1

# Calculate SAND returns
wallet_level_data2['annualized_sand_returns'] = (1+wallet_level_data2['sand_return'])\
                                                    **(365/wallet_level_data2['holding_period_days'])-1

In [ ]:
wallet_level_data2

## Calculate wallet beta

In [ ]:
wallet_level_data2['wallet_beta'] = SAND_PRICE_IN_USD/wallet_level_data2['USD_price_at_timestamp']-1
wallet_level_data2

## Calculating volatility

In [ ]:
# SANITY CHECK to see if the dates for the two df (wallet_level_data2 and sand_info_df) align

# From this check, it's evident that Coin Gecko does not have data on the $SAND price
# before 8/14/2021, which means we won't be able to pull the price data for any dates before then.

print("The minimum date in 'wallet_level_data2':\n", wallet_level_data2['initial_dt'].min(), "\n\n")
print("The minimum date in 'sand_info_df' that is pulled from coin gecko is:\n\n", sand_info_df.min())


In [ ]:
volatility_df = []

for i in list(wallet_level_data2['initial_dt']):
    
    # Convert initial_dt date/time to a standard string format
    start_date = si.standard_date(i)
    
    # Get index corresponding with the specified start_date
    index = si.get_index(start_date, sand_info_df)

    # Filter the sand_info_df to have only prices data over the specified time frame
    get_volatility = si.filter_df(index, sand_info_df)

    # Calculate the volatility for SAND over this timeframe
    volatility = si.sand_volatility(get_volatility)
    
    # Append to the volatility_df list
    volatility_df.append(volatility)

wallet_level_data2['sand_volatility'] = pd.DataFrame(volatility_df)


In [ ]:
wallet_level_data2

## Calculate Sharpe Ratio

In [ ]:

# Calculating the Sharpe Ratio, assume risk free rate of 0.025
wallet_level_data2['sharpe_ratio'] = (wallet_level_data2['return_by_wallet']
                                      -RISK_FREE_RATE)/wallet_level_data2['sand_volatility']

wallet_level_data2

In [ ]:
# Graph the distribution of Sharpe Ratio

x_sharpe = wallet_level_data2['sharpe_ratio']
display(x_sharpe.describe())

plt.subplots()
plt.hist(x_sharpe, color='b')
plt.title("Distribution of Sharpe Ratio", fontsize=20)
plt.xlabel('Sharpe Ratio', fontsize=15)
plt.ylabel('Distribution', color='b', fontsize=15)

In [ ]:
# What is the distribution of holding period in days for our wallets?

x_holding_period = wallet_level_data2['holding_period_days'].sort_values()
less_than_30 = wallet_level_data2[wallet_level_data2['holding_period_days'] > 30]['holders_address'].count() #110 count

plt.subplots(figsize=(15,10))
# plt.plot(x, yp, 'b', label = 'daily price')
plt.hist(x_holding_period, bins = 45)
ymin, ymax = plt.ylim()

plt.title("Distribution of Holding Period (Days)", fontsize=20)
plt.xlabel('Holding Period (days)', fontsize=15)
plt.ylabel('Count of Wallets', color='b', fontsize=15)
plt.axvline(x=30, color='red', linestyle='--')
plt.annotate("Wallets less than \n 30 days old", xy =(30, ymax-10), xytext = (30+5,ymax-10), size=12)

In [ ]:
wallet_level_data2.loc[wallet_level_data2['sharpe_ratio'] > 150]

In [ ]:
top_five_wallets = [
    '0x4db1578cf389b1cda470c08e61c406f3e8bc2e7d', # 41
    '0x6bcbe6c086cc66806b0b7b4a53409058d85f61c8', # 56
    '0x896078a63a1878b7fdc8dba468c9a59b94fd7a92', # 72
    '0xb701dec4186c8c7f1a7047b33f9f9a39febb1db1', # 107
    '0xb8700c17bb966ab854a343377ddf6a2ef06630a3'  # 108
]
display(wallet_level_data2[wallet_level_data2['holders_address'].isin(top_five_wallets)])
                               

In [ ]:
# Graph wallet returns

x_std = wallet_level_data2['sand_volatility']
y_returns = wallet_level_data2['return_by_wallet']

plt.subplots(figsize=(15,10))
# plt.plot(x, yp, 'b', label = 'daily price')
plt.scatter(x_std, y_returns, color='r')

plt.title("Sand Volatility vs Wallet returns", fontsize=20)
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Returns', color='b', fontsize=15)

# Plot wallet #41 (0x4db1578cf389b1cda470c08e61c406f3e8bc2e7d), (sand_volatility, return_by_wallet)
arrowprops = dict(arrowstyle = "->",connectionstyle = "angle, angleA = 0, angleB = 120, rad = 10")
plt.annotate("Wallet #41", xy =(2, 705.67),
            xytext = (2.01,670), size=10,
            arrowprops = arrowprops,)

# Plot wallet #56 (0x6bcbe6c086cc66806b0b7b4a53409058d85f61c8), (sand_volatility, return_by_wallet)
arrowprops = dict(arrowstyle = "->",connectionstyle = "angle, angleA = 0, angleB = 120, rad = 10")
plt.annotate("Wallet #56", xy =(2.17, 80.56),
            xytext = (2.15,90), size=10,
            arrowprops = arrowprops,)

# Plot wallet #72 (0xb701dec4186c8c7f1a7047b33f9f9a39febb1db1), (sand_volatility, return_by_wallet)
arrowprops = dict(arrowstyle = "->",connectionstyle = "angle, angleA = 0, angleB = 120, rad = 10")
plt.annotate("Wallet #72", xy =(2.19, 195.76),
            xytext = (2.2,195), size=10,
            arrowprops = arrowprops,)

# Plot wallet #107 (0x896078a63a1878b7fdc8dba468c9a59b94fd7a92), (sand_volatility, return_by_wallet)
arrowprops = dict(arrowstyle = "->",connectionstyle = "angle, angleA = 0, angleB = 120, rad = 10")
plt.annotate("Wallet #107", xy =(2.02, 140.76),
            xytext = (2,150), size=10,
            arrowprops = arrowprops,)

# Plot wallet #108 (0xb8700c17bb966ab854a343377ddf6a2ef06630a3), (sand_volatility, return_by_wallet)
arrowprops = dict(arrowstyle = "->",connectionstyle = "angle, angleA = 0, angleB = 120, rad = 10")
plt.annotate("Wallet #108", xy =(2.12, 75.3),
            xytext = (2.05,80), size=10,
            arrowprops = arrowprops,)

u=2.1     #x-position of the center
v=400    #y-position of the center
a=0.2     #radius on the x-axis
b=350    #radius on the y-axis

t = np.linspace(0, 2*pi, 100)
plt.plot( u+a*np.cos(t) , v+b*np.sin(t) )
plt.grid(color='lightgray',linestyle='--')



In [ ]:
# Graph wallet alpha

x_std = wallet_level_data2['sand_volatility']
y_returns = wallet_level_data2['wallet_alpha']

plt.subplots(figsize=(15,10))
# plt.plot(x, yp, 'b', label = 'daily price')
plt.scatter(x_std, y_returns, color='r')

plt.title("Sand Volatility vs Wallet Alpha", fontsize=20)
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Wallet Alpha', color='b', fontsize=15)

In [ ]:
# Plot Sharpe ratio

x_std = wallet_level_data2['sand_volatility']
y_returns = wallet_level_data2['sharpe_ratio']

plt.subplots(figsize=(15,10))
# plt.plot(x, yp, 'b', label = 'daily price')
plt.scatter(x_std, y_returns, color='r')

plt.title("Sand Volatility vs Sharpe Ratio", fontsize=20)
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Sharpe Ratio', color='b', fontsize=15)

## Data Analysis

In [ ]:
# Graph histogram of the wallet alpha spread
display(wallet_level_data2['wallet_alpha'].describe())

wallet_level_data2.hist('wallet_alpha')
plt.show()

print('Number of wallets with POSITIVE alpha:', len(wallet_level_data2[wallet_level_data2['wallet_alpha']>0]))
print('Number of wallets with NEGATIVE alpha:', len(wallet_level_data2[wallet_level_data2['wallet_alpha']<0]))

In [ ]:
# Sort by top Sharpe Ratios
sort_sharpe = wallet_level_data2.sort_values('sharpe_ratio', ascending=False)
sharpe_only = sort_sharpe.loc[:2,['holders_address', 'sharpe_ratio']]
sharpe_only.head()

In [ ]:
# Sort by top wallet alphas
sort_alpha = wallet_level_data2.sort_values('wallet_alpha', ascending=False)
alpha_only = sort_alpha.loc[:2,['holders_address', 'wallet_alpha']]
alpha_only.head()

In [ ]:
# Sort by top wallet returns
sort_returns = wallet_level_data2.sort_values('return_by_wallet', ascending=False)
returns_only = sort_returns.loc[:2,['holders_address', 'return_by_wallet']]
returns_only.head()

In [ ]:
# Pull transactions data for top performing wallet #41
df_tx.loc[df_tx['holders_address'] == '0x4db1578cf389b1cda470c08e61c406f3e8bc2e7d'].sort_values('date_time', ascending=False)

In [ ]:
# Graph the waterfall graph for Wallet 41
waterfall_41 = si.waterfall_data(df_tx, '0x4db1578cf389b1cda470c08e61c406f3e8bc2e7d')
si.graph_waterfall(waterfall_41, sand_info_df, "Wallet 41")

In [ ]:
# Graph the waterfall graph for Wallet 56
waterfall_56 = si.waterfall_data(df_tx, '0x6bcbe6c086cc66806b0b7b4a53409058d85f61c8')
si.graph_waterfall(waterfall_56, sand_info_df, "Wallet 56")

In [ ]:
# Graph the waterfall graph for Wallet 72
waterfall_72 = si.waterfall_data(df_tx, '0x896078a63a1878b7fdc8dba468c9a59b94fd7a92')
si.graph_waterfall(waterfall_72, sand_info_df, "Wallet 72")

In [ ]:
waterfall_108 = si.waterfall_data(df_tx, '0xb8700c17bb966ab854a343377ddf6a2ef06630a3')
si.graph_waterfall(waterfall_108, sand_info_df, "Wallet 108")

# Distribution of Returns

In [ ]:
### importing price data from other asset classes
btcusd = pd.read_csv('BTC-USD.csv')
ethusd = pd.read_csv('ETH-USD.csv')
sp500 = pd.read_csv('SPY.csv')

btcusd.rename(columns = {'Adj Close':'BTCUSD'}, inplace = True)
btcusd['btcusd_pct_change'] = btcusd['BTCUSD'].pct_change()
ethusd.rename(columns = {'Adj Close':'ETHUSD'}, inplace = True)
ethusd['ethusd_pct_change'] = ethusd['ETHUSD'].pct_change()
sp500.rename(columns = {'Adj Close':'S&P500'}, inplace = True)
sp500['sp500_pct_change'] = sp500['S&P500'].pct_change()

### importing SAND price from sand_info_df
sand_price = sand_info_df[['date','prices']]
sand_price['prices_pct_change'] = sand_price['prices'].pct_change()
sand_price.rename(columns = {'date':'Date'}, inplace = True)

In [ ]:
### merging all the daily percentages together into one dataframe
df_pct_change = pd.merge(btcusd[['Date','btcusd_pct_change']], ethusd[['Date','ethusd_pct_change']], on='Date', how='outer')
df_pct_change = pd.merge(df_pct_change,sp500[['Date','sp500_pct_change']])
df_pct_change['Date'] = pd.to_datetime(df_pct_change['Date'])
df_pct_change = pd.merge(df_pct_change, sand_price[['Date','prices_pct_change']],how='outer')
df_pct_change.rename(columns = {'prices_pct_change':'SAND_pct_change'}, inplace = True)
# display(df_pct_change.head())
display(df_pct_change.describe())

In [ ]:
histo_list = [df_pct_change['btcusd_pct_change'], df_pct_change['ethusd_pct_change'], df_pct_change['sp500_pct_change'], df_pct_change['SAND_pct_change']]

plt.figure(figsize=(12, 8))
plt.rcParams.update({'font.size': 20})
plt.hist(histo_list, bins=20, histtype='bar', density=True)
plt.title('Distribution of returns')
plt.legend(['BTCUSD', 'ETHUSD','S&P500','SAND'])
plt.ylabel('% frequency')
plt.xlabel('Daily Return')
plt.ylim((None,10.5))
plt.show()

In [ ]:
skew = df_pct_change.skew(axis = 0, skipna = True)
skew.sort_values(inplace=True)
plt.rcParams.update({'font.size': 14})
plt.xticks(rotation=90)
plt.title('Skewness of different coins vs the S&P 500')
plt.bar(skew.index, skew.values ,width=0.4)



## Average holding periods of alpha wallets

In [ ]:
print('Positive alpha wallets:', round(wallet_level_data2[wallet_level_data2['wallet_alpha'] > 0]['holding_period_days'].median(),0), 'days')
print('Negative alpha wallets:', round(wallet_level_data2[wallet_level_data2['wallet_alpha'] < 0]['holding_period_days'].median(),0), 'days')